In [7]:
!pip install kaggle
import kaggle
!kaggle datasets download ankitbansal06/retail-orders -f orders.csv

Dataset URL: https://www.kaggle.com/datasets/ankitbansal06/retail-orders
License(s): CC0-1.0




  0%|          | 0.00/200k [00:00<?, ?B/s]
100%|##########| 200k/200k [00:00<00:00, 234kB/s]
100%|##########| 200k/200k [00:00<00:00, 234kB/s]


In [9]:
import zipfile
zip_ref=zipfile.ZipFile('orders.csv.zip')
zip_ref.extractall()
zip_ref.close()

In [24]:
!pip install pandas
!pip install numpy 
import pandas as pd
import numpy as np
df=pd.read_csv('orders.csv')
df.replace(['Not Available', 'unknown'],np.nan,inplace=True)
df.head(5)

,Order Id,Order Date,Ship Mode,Segment,Country,City,State,Postal Code,Region,Category,Sub Category,Product Id,cost price,List Price,Quantity,Discount Percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [25]:
df['Ship Mode'].unique()

array(['Second Class', 'Standard Class', nan, 'First Class', 'Same Day'],
      dtype=object)

In [45]:
#df.columns=df.columns.str.replace(' ','_')
df.head(5)


,order_id,order_date,ship_mode,segment,country,city,state,postal_code,region,category,sub_category,product_id,cost_price,list_price,quantity,discount_percent
0,1,2023-03-01,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Bookcases,FUR-BO-10001798,240,260,2,2
1,2,2023-08-15,Second Class,Consumer,United States,Henderson,Kentucky,42420,South,Furniture,Chairs,FUR-CH-10000454,600,730,3,3
2,3,2023-01-10,Second Class,Corporate,United States,Los Angeles,California,90036,West,Office Supplies,Labels,OFF-LA-10000240,10,10,2,5
3,4,2022-06-18,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Furniture,Tables,FUR-TA-10000577,780,960,5,2
4,5,2022-07-13,Standard Class,Consumer,United States,Fort Lauderdale,Florida,33311,South,Office Supplies,Storage,OFF-ST-10000760,20,20,2,5


In [69]:
#discount price
df['discount']=df['list_price']*df['discount_percent']*0.01
#sales price
df['sale_price']=df['list_price']-df['discount']
#profit
df['profit']=df['sale_price']-df['cost_price']
#drop columns
df.drop(columns=['list_price','discount_percent','cost_price'],inplace=True)
df.dtypes

order_id                 int64
order_date      datetime64[ns]
ship_mode               object
segment                 object
country                 object
city                    object
state                   object
postal_code              int64
region                  object
category                object
sub_category            object
product_id              object
quantity                 int64
discount               float64
sale_price             float64
profit                 float64
dtype: object

In [63]:
df['order_date']=pd.to_datetime(df['order_date'],format="%Y-%m-%d")
df.dtypes

order_id                 int64
order_date      datetime64[ns]
ship_mode               object
segment                 object
country                 object
city                    object
state                   object
postal_code              int64
region                  object
category                object
sub_category            object
product_id              object
quantity                 int64
discount               float64
sale_price             float64
profit                 float64
dtype: object

In [75]:
!pip install pyodbc

   ---------------------------------------- 0.0/69.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/69.3 kB ? eta -:--:--
   ---------------------------------------- 0.0/69.3 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/69.3 kB ? eta -:--:--
   ----------------- ---------------------- 30.7/69.3 kB 330.3 kB/s eta 0:00:01
   ----------------- ---------------------- 30.7/69.3 kB 330.3 kB/s eta 0:00:01
   ----------------------- ---------------- 41.0/69.3 kB 164.3 kB/s eta 0:00:01
   ---------------------------------------- 69.3/69.3 kB 290.7 kB/s eta 0:00:00


In [76]:
import sqlalchemy as sal

# Use a raw string or escape backslashes
engine = sal.create_engine(r'mssql+pyodbc://LAPTOP-O0VTSVD8\SQLEXPRESS/master?driver=ODBC+DRIVER+17+FOR+SQL+SERVER')
conn = engine.connect()


In [81]:
df.to_sql('df_orders', con=conn , index=False, if_exists = 'append')

38